<a href="https://colab.research.google.com/github/yashugupta786/my_codes-and-project/blob/master/word_2vec_shape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import itertools
import pandas as pd
import numpy as np
import re
import os
# from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

# # Deep learning: 
# from tensorflow.keras.models import Input
# from tensorflow.keras.layers import Dense

from scipy import sparse

# Custom functions
from utility import text_preprocessing,create_unique_word_dict

In [63]:
texts1 = pd.read_csv('sample.csv')
texts2 = [x for x in texts1['text']]

# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists = []
all_text = []



In [64]:

texts=texts2[0:2]

In [65]:
for text in texts:

    # Cleaning the text
    text = text_preprocessing(text)

    # Appending to the all text list
    all_text += text 

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words    
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])


In [66]:
word_lists

[['future', 'king'],
 ['future', 'prince'],
 ['king', 'prince'],
 ['king', 'future'],
 ['prince', 'king'],
 ['prince', 'future'],
 ['daughter', 'princess'],
 ['princess', 'daughter']]

In [67]:
unique_word_dict = create_unique_word_dict(all_text)
unique_word_dict

{'daughter': 0, 'future': 1, 'king': 2, 'prince': 3, 'princess': 4}

In [68]:
n_words = len(unique_word_dict)
print(n_words)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

5


In [69]:
for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)


8it [00:00, 11785.89it/s]


In [70]:
X,Y

([array([0., 1., 0., 0., 0.]),
  array([0., 1., 0., 0., 0.]),
  array([0., 0., 1., 0., 0.]),
  array([0., 0., 1., 0., 0.]),
  array([0., 0., 0., 1., 0.]),
  array([0., 0., 0., 1., 0.]),
  array([1., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 1.])],
 [array([0., 0., 1., 0., 0.]),
  array([0., 0., 0., 1., 0.]),
  array([0., 0., 0., 1., 0.]),
  array([0., 1., 0., 0., 0.]),
  array([0., 0., 1., 0., 0.]),
  array([0., 1., 0., 0., 0.]),
  array([0., 0., 0., 0., 1.]),
  array([1., 0., 0., 0., 0.])])

In [71]:
X = sparse.csr_matrix(X)
Y = sparse.csr_matrix(Y)

In [72]:
X.shape

(8, 5)

In [73]:
embed_size = 2

# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.summary()
# Optimizing the network weights
# model.fit(
#     x=X, 
#     y=Y, 
#     batch_size=256,
#     epochs=100)


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 12        
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 15        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
